# Business Problem

<font size="4"> The problem that we are going to undertake here is of a restaurant location. In any city, restaurants are a part of the spirit and night life of the city, socially and culturally. In this report we focusing our attention to Downtown Toronto, a Borough of the largest city in Canada. Our focus is on finding the best neghborhood to open a restaurant in Downtown, Toronto. We have been tasked to find a optimum location for a restaurant to flourish, and this help our target audience of restauranteer that are loking to expand their franchise, or emerging one's that have just started food as a potential business of the future.</font>

## Process

<font size = "4"> The major variables in any restaurant locations are population and frequency of their visit to a restaurant. These two variables will be the deciding criterion to select a location for opening a restaurant. </font>

<font size = "4"> We are going to use the clustering method to understand, which neighborhood of Downtown, Toronto,based on the aforementioned variables. </font>

# What Data is Required ?

<font size = "4"> Main obstacle to any data science problem is collection of data, for the specified problem. For the problem we are focusin on, we need to look for the data, that have locations involved in it. We need to look specifically for restaurants and cafe's in any neighborhood, frequency of visits. </font>

# DATA

<font size = "4"> Our major source of Data is Foursquare. From their we collect the location data of different neighborhoods, focus mailny on cafe's and restaurants. We will be using Places Api to make some calls to collect the data for each neighborhood, and tabulate it in the dataframe. ut, before we start to collect the neighborhood information of Downtown, Toronto. That data has been collected from the wikipedia, postal codes. </font>

# code

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
!pip install selenium
!pip install BeautifulSoup4
from selenium import webdriver
from bs4 import BeautifulSoup

     |████████████████████████████████| 911kB 12.8MB/s eta 0:00:01


In [3]:
!pip install requests

In [4]:
import requests
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find_all('table')
df = pd.read_html(str(table))[0] 
Code = []
Location = []
for i in range(0,9):
    for j in range(0,20):
        string = str(df.loc[j,i])
        Code.append(string[:3])
        Location.append(string[3:])
        
df = pd.DataFrame()
df['PostalCode'] = Code
df['B'] = Location

df_ = df[df['B'] != 'Not assigned']
df2 = df_.reset_index()
df2.drop('index', axis = 1, inplace = True)

In [6]:
Borough = []
neighborhood = []
Neighborhood = []
for i in range(0,103):
    string_ = df2.iloc[i,1]
    indx = string_.find('(')
    Borough.append(string_[:indx])
    neighborhood.append(string_[indx+1:len(string_)])
    
for word in neighborhood:
    word = word.replace(')', ' ')
    word = word.replace('(', '')
    word = word.replace(' /', ',')
    Neighborhood.append(word)

df2['Borough'] = Borough
df2['Neighborhood'] = Neighborhood
df3 = df2.drop('B', axis = 1)
df3 = df3.drop('PostalCode', axis = 1)

In [7]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
df3['Latitude'] = coordinates['Latitude']
df3['Longitude'] = coordinates['Longitude']
Down = df3[df3['Borough'] == 'Downtown Toronto']

In [53]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Down['Latitude'], Down['Longitude'], Down['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

    
map_toronto

Borough         Downtown Toronto
Neighborhood           Rosedale 
Latitude                 43.6796
Longitude               -79.3775
Name: 50, dtype: object

In [38]:
CLIENT_ID = '5XLBNHIEOIVYDD5JNEH2HDL2QRIMTKB0XQJ2CN5XM41FEEU0' 
CLIENT_SECRET = 'RYGDPE41CGJEZNCWVWSV2TDHVWMEWDN1HWVB1VZFZZDNHHKK' 
VERSION = '20180605'
LIMIT = 100
radius = 500
category = '4d4b7105d754a06374d81259'
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Down.loc[50, 'Latitude'], 
    Down.loc[50, 'Longitude'], 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [77]:
results['response']['venues'][0]

{'id': '50f72075e4b0507e7ae0a310',
 'name': 'Home Coffee Solutions',
 'location': {'address': '385 Front St West',
  'crossStreet': 'East of Spadina',
  'lat': 43.67596,
  'lng': -79.38137,
  'labeledLatLngs': [{'label': 'display', 'lat': 43.67596, 'lng': -79.38137}],
  'distance': 506,
  'postalCode': 'M4W 3Y3',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['385 Front St West (East of Spadina)',
   'Toronto ON M4W 3Y3',
   'Canada']},
 'categories': [{'id': '4bf58dd8d48988d1e0931735',
   'name': 'Coffee Shop',
   'pluralName': 'Coffee Shops',
   'shortName': 'Coffee Shop',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1585393474',
 'hasPerk': False}

In [85]:
venues = []

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryID={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            category)
            
        
        results = requests.get(url).json()["response"]
        
        for i in range(len(results['venues'])):
            venues.append([name, lat, lng, results['venues'][i]['name'], results['venues'][i]['location']['lat'], 
            results['venues'][i]['location']['lng'],  
            results['venues'][i]['categories'][0]['name']])
    
    return(venues)

In [86]:
Downtown_venues = getNearbyVenues(names=Down['Neighborhood'],
                                   latitudes=Down['Latitude'],
                                   longitudes=Down['Longitude']
                                  )
Downtown_venues

Rosedale 
St. James Town, Cabbagetown 
Church and Wellesley 
Regent Park, Harbourfront 
Garden District, Ryerson 
St. James Town 
Berczy Park 
Central Bay Street 
Richmond, Adelaide, King 
Harbourfront East, Union Station, Toronto Islands 
Toronto Dominion Centre, Design Exchange 
Commerce Court, Victoria Hotel 
University of Toronto, Harbord 
Kensington Market, Chinatown, Grange Park 
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport 
First Canadian Place, Underground city 
Christie 


[['Rosedale ',
  43.6795626,
  -79.37752940000001,
  'Home Coffee Solutions',
  43.67596,
  -79.38137,
  'Coffee Shop'],
 ['Rosedale ',
  43.6795626,
  -79.37752940000001,
  "Jack Astor's Band of Chefs Food Truck",
  43.67592238709114,
  -79.38195852454088,
  'Food Truck'],
 ['Rosedale ',
  43.6795626,
  -79.37752940000001,
  'Bacaro',
  43.675463,
  -79.382403,
  'Italian Restaurant'],
 ['St. James Town, Cabbagetown ',
  43.667967,
  -79.3676753,
  'Suruthis Take Out',
  43.667215255077906,
  -79.36936078482839,
  'Sri Lankan Restaurant'],
 ['St. James Town, Cabbagetown ',
  43.667967,
  -79.3676753,
  'Staij & Co Café and Pastry Boutique',
  43.666862703979966,
  -79.36922818422318,
  'Café'],
 ['St. James Town, Cabbagetown ',
  43.667967,
  -79.3676753,
  'Butter Chicken Factory',
  43.66707247004843,
  -79.36918422958169,
  'Indian Restaurant'],
 ['St. James Town, Cabbagetown ',
  43.667967,
  -79.3676753,
  'Cranberries',
  43.6678427705951,
  -79.36940687874281,
  'Diner'],
 ['St

In [87]:
DownToro = DownToro = pd.DataFrame(Downtown_venues ,columns = ['Neighborhood', 'Neighbor Lat', 'Neighbor Long', 'Venue Name', 'Venue Lat', 'Venue Long', 'Venue Type'])
DownToro.head()

,Neighborhood,Neighbor Lat,Neighbor Long,Venue Name,Venue Lat,Venue Long,Venue Type
0,Rosedale,43.679563,-79.377529,Home Coffee Solutions,43.675960,-79.381370,Coffee Shop
1,Rosedale,43.679563,-79.377529,Jack Astor's Band of Chefs Food Truck,43.675922,-79.381959,Food Truck
2,Rosedale,43.679563,-79.377529,Bacaro,43.675463,-79.382403,Italian Restaurant
3,"St. James Town, Cabbagetown",43.667967,-79.367675,Suruthis Take Out,43.667215,-79.369361,Sri Lankan Restaurant
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Staij & Co Café and Pastry Boutique,43.666863,-79.369228,Café


In [89]:
DownToro.groupby('Neighborhood').count()

,Neighbor Lat,Neighbor Long,Venue Name,Venue Lat,Venue Long,Venue Type
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",4,4,4,4,4,4
Central Bay Street,50,50,50,50,50,50
Christie,27,27,27,27,27,27
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",50,50,50,50,50,50
"Harbourfront East, Union Station, Toronto Islands",50,50,50,50,50,50


In [90]:
print('There are {} restaurants in Downtown.'.format(len(DownToro['Venue Name'].unique())))


There are 405 restaurants in Downtown.


In [93]:
Downtown_onehot = pd.get_dummies(DownToro[['Venue Type']], prefix="", prefix_sep="")

Downtown_onehot['Neighborhood'] = DownToro['Neighborhood']

fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Creperie,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Halal Restaurant,Hot Dog Joint,Hotel Bar,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Noodle House,Office,Pizza Place,Poke Place,Poutine Place,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Spanish Restaurant,Speakeasy,Sports Bar,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,"St. James Town, Cabbagetown",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Beer Bar,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Convenience Store,Creperie,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Dumpling Restaurant,Empanada Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Gourmet Shop,Greek Restaurant,Grocery Store,Halal Restaurant,Hot Dog Joint,Hotel Bar,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Korean Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Noodle House,Office,Pizza Place,Poke Place,Poutine Place,Pub,Ramen Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Spanish Restaurant,Speakeasy,Sports Bar,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.00,0.000000,0.040000,0.020000,0.020000,0.00,0.020000,0.02,0.02,0.02,0.040000,0.00,0.040000,0.00,0.000000,0.000000,0.00,0.00,0.02,0.04,0.020000,0.000000,0.000000,0.000000,0.000000,0.00,0.280000,0.00,0.00,0.00,0.040000,0.000000,0.00,0.020000,0.00,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.020000,0.00,0.000000,0.02,0.000000,0.02,0.000000,0.060000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.040000,0.000000,0.060000,0.00,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.000000,0.00,0.020000,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.25,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.25,0.000000,0.00,0.00,0.00,0.00,0.00
2,Central Bay Street,0.00,0.000000,0.020000,0.020000,0.000000,0.00,0.020000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.060000,0.000000,0.04,0.00,0.00,0.00,0.040000,0.000000,0.000000,0.020000,0.000000,0.00,0.220000,0.00,0.00,0.00,0.000000,0.020000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.06,0.00,0.08,0.02,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.00,0.060000,0.060000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.02,0.00,0.000000,0.000000,0.080000,0.00,0.020000,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.040000,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.00
3,Christie,0.00,0.000000,0.037037,0.000000,0.000000,0.00,0.148148,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.259259,0.037037,0.000000,0.000000,0.000000,0.00,0.074074,0.00,0.00,0.00,0.000000,0.000000,0.00,0.037037,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.037037,0.00,0.037037,0.00,0.000000,0.00,0.074074,0.037037,0.00,0.037037,0.037037,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.074074,0.00,0.037037,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.037037,0.0

In [97]:
for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head())
    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0: num_top_venues]

----Berczy Park ----
                 venue  freq
0          Coffee Shop  0.28
1  Japanese Restaurant  0.06
2           Restaurant  0.06
3               Bistro  0.04
4        Deli / Bodega  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport ----
                 venue  freq
0          Coffee Shop  0.25
1  American Restaurant  0.25
2                  Bar  0.25
3     Tapas Restaurant  0.25
4    Korean Restaurant  0.00


----Central Bay Street ----
                 venue  freq
0          Coffee Shop  0.22
1           Food Court  0.08
2           Restaurant  0.08
3   Italian Restaurant  0.06
4  Japanese Restaurant  0.06


----Christie ----
                venue  freq
0                Café  0.26
1              Bakery  0.15
2          Restaurant  0.07
3  Italian Restaurant  0.07
4         Coffee Shop  0.07


----Church and Wellesley ----
                  venue  freq
0           Coffee Shop  0.24
1  Fast Food Restaurant  0.10

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Japanese Restaurant,Breakfast Spot,American Restaurant,Bistro,Pub,Deli / Bodega,Cafeteria,Café
1,"CN Tower, King and Spadina, Railway Lands, Har...",American Restaurant,Coffee Shop,Tapas Restaurant,Bar,Wine Bar,Diner,Cocktail Bar,Comfort Food Restaurant,Convenience Store,Creperie
2,Central Bay Street,Coffee Shop,Food Court,Restaurant,Italian Restaurant,Japanese Restaurant,Fast Food Restaurant,Bubble Tea Shop,Café,Burger Joint,Sushi Restaurant
3,Christie,Café,Bakery,Restaurant,Italian Restaurant,Coffee Shop,Thai Restaurant,American Restaurant,Korean Restaurant,Sandwich Place,Juice Bar
4,Church and Wellesley,Coffee Shop,Fast Food Restaurant,Restaurant,Breakfast Spot,Bubble Tea Shop,Café,Dessert Shop,Gastropub,Halal Restaurant,Pizza Place
5,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Food Court,American Restaurant,Bar,Café,Fast Food Restaurant,Italian Restaurant,Gastropub,Bakery
6,"First Canadian Place, Underground city",Coffee Shop,Restaurant,Food Court,American Restaurant,Bar,Café,Fast Food Restaurant,Gastropub,Office,Deli / Bodega
7,"Garden District, Ryerson",Coffee Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop,Italian Restaurant,Ramen Restaurant,Gastropub,Food Court,American Restaurant,Fast Food Restaurant
8,"Harbourfront East, Union Station, Toronto Isla...",Coffee Shop,Restaurant,Café,Sandwich Place,Sports Bar,Ramen Restaurant,Japanese Restaurant,American Restaurant,Food Court,Hotel Bar
9,"Kensington Market, Chinatown, Grange Park",Coffee Shop,Chinese Restaurant,Café,Bubble Tea Shop,Vietnamese Restaurant,Dumpling Restaurant,Fast Food Restaurant,Noodle House,Pizza Place,Ramen Restaurant


In [100]:
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

kmeans.labels_[0:10]

array([0, 3, 0, 4, 0, 0, 0, 0, 0, 2], dtype=int32)

In [101]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = Down

Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Food Truck,Coffee Shop,Italian Restaurant,Wine Bar,Dim Sum Restaurant,Chocolate Shop,Cocktail Bar,Comfort Food Restaurant,Convenience Store,Creperie
51,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Café,Restaurant,Breakfast Spot,Pizza Place,Chinese Restaurant,Japanese Restaurant,Gastropub,BBQ Joint,Thai Restaurant
52,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Fast Food Restaurant,Restaurant,Breakfast Spot,Bubble Tea Shop,Café,Dessert Shop,Gastropub,Halal Restaurant,Pizza Place
53,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Italian Restaurant,Thai Restaurant,Café,Bakery,Breakfast Spot,African Restaurant,Mediterranean Restaurant,Pizza Place,Deli / Bodega
54,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Japanese Restaurant,Restaurant,Bubble Tea Shop,Italian Restaurant,Ramen Restaurant,Gastropub,Food Court,American Restaurant,Fast Food Restaurant


In [102]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [103]:
for i in range(0,5):
    print("Cluster {}:".format(i))
    c = Downtown_merged.loc[Downtown_merged['Cluster Labels'] == i]
    g = Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]
    print(c, g)

Cluster 0:
             Borough                                       Neighborhood  \
52  Downtown Toronto                              Church and Wellesley    
54  Downtown Toronto                          Garden District, Ryerson    
55  Downtown Toronto                                    St. James Town    
56  Downtown Toronto                                       Berczy Park    
57  Downtown Toronto                                Central Bay Street    
58  Downtown Toronto                          Richmond, Adelaide, King    
59  Downtown Toronto  Harbourfront East, Union Station, Toronto Isla...   
60  Downtown Toronto          Toronto Dominion Centre, Design Exchange    
61  Downtown Toronto                    Commerce Court, Victoria Hotel    
70  Downtown Toronto            First Canadian Place, Underground city    

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
52  43.665860 -79.383160               0           Coffee Shop   
54  43.657162 -79.378937       

In [104]:
N = DownToro.groupby('Neighborhood').count()
N.sort_values('Venue Name', axis = 0, ascending = True, inplace =True)
N

,Neighbor Lat,Neighbor Long,Venue Name,Venue Lat,Venue Long,Venue Type
Neighborhood,,,,,,
Rosedale,3,3,3,3,3,3
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",4,4,4,4,4,4
Christie,27,27,27,27,27,27
"St. James Town, Cabbagetown",48,48,48,48,48,48
"Regent Park, Harbourfront",49,49,49,49,49,49
Berczy Park,50,50,50,50,50,50
St. James Town,50,50,50,50,50,50
"Richmond, Adelaide, King",50,50,50,50,50,50
"Kensington Market, Chinatown, Grange Park",50,50,50,50,50,50


# Methodology

<font size = 4> In this analysis, we have used foursquare to get the data of restaurants in the borough Downtown of city of Toronto, Ontario, Canada. With the help of clustering , that groups different datas, based on their similarities, into one cluster and dissimilar data into different clusters.</font>

# Result

<font size = 4> We have found through our analysis is that the most common food places is in Downtown, Toronto is are coffee shops, and second most common are fast food restaurents. Also it can be noted that the neighborhood with least number of restaurants of coffee places is Riverdale, followed by CN towers, Island Airport....  So, as per the data, any restautanteer shoul open a coffee shops as to benefit from the frequency of visits, and open in Riverdale or Island Airport, as to face minimum competetion from other establishments.</font>